[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nopalraihan/Data-Porto/blob/claude/document-automation-excel-pdf-QC9W8/document_automation/Document_Automation_PLN_Crosscheck.ipynb)

# PLN Document Automation & Crosscheck System

**ML-Powered document verification pipeline**

This notebook provides:
1. **PLN Document Crosscheck** — Read PDF doc → compare against Excel template → verify or log mismatches
2. **General Document Automation** — Read any CSV/Excel/PDF → process → export to formatted Excel & PDF reports
3. **ML Analysis** — TF-IDF text similarity, Anomaly Detection, Random Forest confidence scoring

---

### How to use
1. Click the **"Open in Colab"** badge above to launch in Google Colab
2. Run **Section 1** to install dependencies & clone the repo
3. Run **Section 2** for the PLN Crosscheck demo (or upload your own files)
4. Run **Section 3** for general document automation

> Works on **Google Colab** and any Jupyter environment.

---
## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q pandas openpyxl fpdf2 PyPDF2 scikit-learn

In [ ]:
# Clone the repository (skip if running locally)
import os

REPO_URL = "https://github.com/nopalraihan/Data-Porto.git"
BRANCH = "claude/document-automation-excel-pdf-QC9W8"
PROJECT_DIR = "/content/Data-Porto"  # Colab path

# Use local path if not on Colab
try:
    from google.colab import drive
    ON_COLAB = True
except ImportError:
    ON_COLAB = False
    PROJECT_DIR = os.path.dirname(os.path.abspath("__file__"))
    # Walk up to find Data-Porto root
    while PROJECT_DIR != "/" and not os.path.isfile(os.path.join(PROJECT_DIR, "requirements.txt")):
        PROJECT_DIR = os.path.dirname(PROJECT_DIR)

if ON_COLAB:
    if os.path.isdir(PROJECT_DIR):
        print(f"Repo already exists at {PROJECT_DIR}")
        !cd {PROJECT_DIR} && git pull origin {BRANCH}
    else:
        !git clone -b {BRANCH} {REPO_URL} {PROJECT_DIR}

# Add project root to Python path
import sys
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

print(f"Project directory: {PROJECT_DIR}")
print(f"Running on: {'Google Colab' if ON_COLAB else 'Local Jupyter'}")

In [ ]:
# Verify imports work
from document_automation.readers.document_reader import DocumentReader
from document_automation.writers.excel_writer import ExcelWriter
from document_automation.writers.pdf_writer import PDFWriter
from document_automation.utils.data_processor import DataProcessor
from document_automation.pipeline import Pipeline
from document_automation.crosscheck.pln_extractor import PLNExtractor
from document_automation.crosscheck.crosscheck_engine import CrosscheckEngine
from document_automation.crosscheck.report_generator import ReportGenerator
from document_automation.crosscheck.create_template import create_template
from document_automation.ml.text_similarity import TextSimilarity
from document_automation.ml.anomaly_detector import AnomalyDetector
from document_automation.ml.confidence_scorer import ConfidenceScorer

import pandas as pd
import numpy as np

print("All modules imported successfully!")

---
## 2. PLN Document Crosscheck

### Flow:
```
PDF Document (from field)     Excel Template (from HO PLN)
        |                              |
        v                              v
  PLNExtractor                   pd.read_excel()
  (extract fields)              (load expected data)
        |                              |
        +----------+   +---------------+
                   v   v
            CrosscheckEngine + ML Models
                     |
          +----------+----------+
          v          v          v
      ALL MATCH   MISMATCH   MISSING
      VERIFIED    error log  warning
                     |
                     v
              ReportGenerator
         (Excel + PDF output reports)
```

### 2a. Run Demo (with sample data)
This demo simulates a PLN document with **2 intentional mismatches** to show the full verification flow.

In [ ]:
# Create output directory
OUTPUT_DIR = os.path.join(PROJECT_DIR, "document_automation", "output")
SAMPLE_DIR = os.path.join(PROJECT_DIR, "document_automation", "sample_data")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SAMPLE_DIR, exist_ok=True)

# Step 1: Create the Excel template
template_path = create_template(os.path.join(SAMPLE_DIR, "PLN_Crosscheck_Template.xlsx"))
print(f"Template created: {template_path}")

# Preview the template
df_template = pd.read_excel(template_path, sheet_name="Data Pelanggan", header=3)
df_template

In [ ]:
# Step 2: Simulate PDF extraction
# (In real use, PLNExtractor reads this from your actual PDF)
simulated_pdf_fields = {
    "id_pelanggan": "532100012345",
    "nama_pelanggan": "SUHARTO",
    "alamat": "JL. PENGGILINGAN ELOK NO.23 RT005/RW012, PENGGILINGAN, CAKUNG, JAKARTA TIMUR",
    "tarif_daya": "R1/1300 VA",
    "nomor_meter": "JTX476",
    "nomor_kwh": "85201234",
    "periode": "Januari 2026",
    "stand_meter_awal": "15230",
    "stand_meter_akhir": "15510",     # << MISMATCH: Excel says 15480
    "pemakaian_kwh": "280",            # << MISMATCH: Excel says 250
    "biaya_listrik": "352500",
}

print("Simulated PDF fields:")
for k, v in simulated_pdf_fields.items():
    print(f"  {k:25s} = {v}")

In [ ]:
# Step 3: Run the ML-enhanced crosscheck
engine = CrosscheckEngine(simulated_pdf_fields, df_template, use_ml=True)
result = engine.run()

# Display results as a table
results_df = pd.DataFrame(result["results"])
display_cols = ["field_name", "pdf_value", "excel_value", "match_status", "notes"]
if "similarity_score" in results_df.columns:
    display_cols.append("similarity_score")

print("\n=== CROSSCHECK RESULTS ===")
results_df[display_cols]

In [ ]:
# Step 4: View summary & ML analysis
summary = result["summary"]
pct = summary["match_percentage"]

print("=" * 50)
if pct == 100:
    print(f"VERDICT: VERIFIED - All {summary['total_fields_checked']} fields match!")
else:
    print(f"VERDICT: MISMATCH DETECTED")
    print(f"  {summary['total_match']}/{summary['total_fields_checked']} match ({pct}%)")
    print(f"  {summary['total_mismatch']} MISMATCHES | {summary['total_missing']} MISSING")
print("=" * 50)

# ML Confidence
if "ml_confidence" in result:
    conf = result["ml_confidence"]
    print(f"\nML Confidence Score: {conf['confidence_score']}%")
    print(f"Prediction: {conf['prediction']}")
    print(f"Risk Level: {conf['risk_level']}")

# Text Similarity
if "ml_similarity" in result:
    print(f"\nText Similarity:")
    for field, sim in result["ml_similarity"].items():
        print(f"  {field}: {sim['score']:.4f} ({sim['classification']})")

# Anomalies
if "ml_anomalies" in result:
    flags = result["ml_anomalies"]
    if flags:
        print(f"\nAnomalies ({len(flags)}):")
        for f in flags:
            print(f"  [{f['severity']}] {f['field']}: {f['message']}")
    else:
        print("\nNo anomalies detected.")

# Error log
mismatches = [r for r in result["results"] if r["match_status"] == "MISMATCH"]
if mismatches:
    print(f"\nERROR LOG ({len(mismatches)} mismatches):")
    for i, r in enumerate(mismatches, 1):
        print(f"  #{i} {r['field_name']}")
        print(f"     PDF:   {r.get('pdf_value', 'N/A')}")
        print(f"     Excel: {r.get('excel_value', 'N/A')}")

In [ ]:
# Step 5: Generate Excel & PDF reports
reporter = ReportGenerator(
    result,
    {"file_name": "23.Dok PLN PENGGILINGANELOK_JTX476.pdf", "page_count": 2},
    OUTPUT_DIR,
)
outputs = reporter.generate_all()

print(f"Excel report: {outputs['excel']}")
print(f"PDF report:   {outputs['pdf']}")

# Download files on Colab
if ON_COLAB:
    from google.colab import files
    print("\nDownloading reports...")
    files.download(outputs["excel"])
    files.download(outputs["pdf"])

### 2b. Upload Your Own Files
Upload your actual PLN PDF and Excel template to crosscheck.

In [ ]:
# Upload files (Colab only - skip on local Jupyter)
pdf_path = None
excel_path = None

if ON_COLAB:
    from google.colab import files
    print("Upload your PLN PDF document:")
    uploaded = files.upload()
    for name in uploaded:
        if name.lower().endswith(".pdf"):
            pdf_path = os.path.join("/content", name)
            with open(pdf_path, "wb") as f:
                f.write(uploaded[name])
            print(f"  PDF saved: {pdf_path}")

    print("\nUpload your Excel template:")
    uploaded = files.upload()
    for name in uploaded:
        if name.lower().endswith((".xlsx", ".xls")):
            excel_path = os.path.join("/content", name)
            with open(excel_path, "wb") as f:
                f.write(uploaded[name])
            print(f"  Excel saved: {excel_path}")
else:
    # For local Jupyter: set paths manually
    # pdf_path = "/path/to/your/PLN_document.pdf"
    # excel_path = "/path/to/your/PLN_Crosscheck_Template.xlsx"
    print("Local mode: Set pdf_path and excel_path manually in the cell above.")

In [ ]:
# Run crosscheck on your uploaded files
if pdf_path and excel_path:
    # Extract from PDF
    print("Extracting fields from PDF...")
    extractor = PLNExtractor(pdf_path)
    extraction = extractor.extract()
    pdf_fields = {k: v["value"] for k, v in extraction["fields"].items()}

    print(f"Found {len(pdf_fields)} fields:")
    for k, v in pdf_fields.items():
        print(f"  {k:25s} = {v}")

    # Load template
    df = pd.read_excel(excel_path, sheet_name="Data Pelanggan", header=3)
    print(f"\nTemplate rows: {len(df)}")

    # Crosscheck
    print("\nRunning crosscheck...")
    engine = CrosscheckEngine(pdf_fields, df, use_ml=True)
    result = engine.run()

    # Show results table
    results_df = pd.DataFrame(result["results"])
    display(results_df[["field_name", "pdf_value", "excel_value", "match_status", "notes"]])

    # Verdict
    s = result["summary"]
    print(f"\nMatch: {s['total_match']}/{s['total_fields_checked']} ({s['match_percentage']}%)")
    if "ml_confidence" in result:
        print(f"ML Confidence: {result['ml_confidence']['confidence_score']}% ({result['ml_confidence']['prediction']})")

    # Generate reports
    reporter = ReportGenerator(result, extraction["metadata"], OUTPUT_DIR)
    outputs = reporter.generate_all()
    print(f"\nExcel: {outputs['excel']}")
    print(f"PDF:   {outputs['pdf']}")

    if ON_COLAB:
        files.download(outputs["excel"])
        files.download(outputs["pdf"])
else:
    print("No files uploaded. Run the upload cell above first, or use the demo in Section 2a.")

---
## 3. General Document Automation

Read any **CSV**, **Excel**, or **PDF** file → process → export to formatted **Excel** and **PDF** reports.

### 3a. Using the sample employee dataset

In [ ]:
# Run pipeline on sample CSV
sample_csv = os.path.join(PROJECT_DIR, "document_automation", "sample_data", "employees.csv")

pipeline = Pipeline(sample_csv, output_dir=OUTPUT_DIR)
result = pipeline.run(
    title="Employee Analytics Report",
    filters=[{"column": "salary", "operator": ">=", "value": 60000}],
    group_by={"group_col": "department", "agg_col": "salary", "agg_func": "mean"},
    top_n={"column": "salary", "n": 10, "ascending": False},
)

print(f"Excel: {result['excel']}")
print(f"PDF:   {result['pdf']}")
print(f"Summary: {result['summary']}")

if ON_COLAB:
    files.download(result["excel"])
    files.download(result["pdf"])

### 3b. Upload your own document

In [ ]:
# Upload any CSV, Excel, or PDF file
doc_path = None

if ON_COLAB:
    from google.colab import files
    print("Upload a CSV, Excel, or PDF file:")
    uploaded = files.upload()
    for name in uploaded:
        doc_path = os.path.join("/content", name)
        with open(doc_path, "wb") as f:
            f.write(uploaded[name])
        print(f"Saved: {doc_path}")
else:
    # doc_path = "/path/to/your/file.csv"
    print("Local mode: Set doc_path manually.")

In [ ]:
# Process the uploaded document
if doc_path:
    # Preview
    reader = DocumentReader(doc_path)
    print(reader.summary())
    print()

    # Run pipeline
    pipeline = Pipeline(doc_path, output_dir=OUTPUT_DIR)
    result = pipeline.run(title="Document Report")

    for key, val in result.items():
        print(f"{key}: {val}")

    if ON_COLAB:
        if "excel" in result:
            files.download(result["excel"])
        if "pdf" in result:
            files.download(result["pdf"])
else:
    print("No file uploaded. Run the upload cell above first.")

---
## 4. ML Models - Standalone Usage

Use the ML components independently.

In [ ]:
# TF-IDF Text Similarity
sim = TextSimilarity()

test_pairs = [
    ("JL. PENGGILINGAN ELOK NO.23", "JALAN PENGGILINGAN ELOK NOMOR 23"),
    ("SUHARTO", "SUHARTO"),
    ("DEWI SARTIKA", "DWI SARTIKA"),
    ("Jl. Merdeka No.10 Jakarta", "Jalan Merdeka Nomor 10, Jakarta Pusat"),
    ("BAMBANG WIJAYA", "AHMAD FAUZI"),
]

print("TF-IDF Cosine Similarity Results:")
print("-" * 80)
for a, b in test_pairs:
    score = sim.score(a, b)
    cls = sim.classify_match(score)
    print(f"  {score:.4f} [{cls:10s}]  '{a}'  vs  '{b}'")

In [ ]:
# Anomaly Detection
detector = AnomalyDetector()

# Normal case
normal = {
    "tarif_daya": "R1/1300 VA",
    "stand_meter_awal": "15230",
    "stand_meter_akhir": "15480",
    "pemakaian_kwh": "250",
    "biaya_listrik": "361000",
}

# Suspicious case
suspicious = {
    "tarif_daya": "R1/900 VA",
    "stand_meter_awal": "10000",
    "stand_meter_akhir": "9500",   # meter went backwards!
    "pemakaian_kwh": "5000",        # impossibly high
    "biaya_listrik": "100000",      # rate doesn't match
}

print("Normal case:")
flags = detector.check_single(normal)
print(f"  {len(flags)} flags" if flags else "  No anomalies")

print("\nSuspicious case:")
flags = detector.check_single(suspicious)
for f in flags:
    print(f"  [{f['severity']:8s}] {f['field']}: {f['message']}")
    print(f"           Expected: {f['expected']} | Actual: {f['actual']}")

In [ ]:
# Random Forest Confidence Scorer
scorer = ConfidenceScorer()
train_info = scorer.train()
print("Model trained:")
print(f"  Accuracy: {train_info['train_accuracy']:.2%}")
print(f"  Samples: {train_info['samples']}")
print(f"\nFeature importances:")
for feat, imp in sorted(train_info["feature_importances"].items(), key=lambda x: -x[1]):
    bar = '#' * int(imp * 50)
    print(f"  {feat:25s} {imp:.4f}  {bar}")

# Score a valid document
print("\n--- Valid document ---")
r = scorer.score({"match_ratio": 1.0, "name_similarity": 0.98, "address_similarity": 0.95,
                  "meter_deviation": 0.0, "billing_deviation": 0.0, "anomaly_count": 0, "missing_fields": 0})
print(f"  Confidence: {r['confidence_score']}% | {r['prediction']} | Risk: {r['risk_level']}")

# Score a suspicious document
print("\n--- Suspicious document ---")
r = scorer.score({"match_ratio": 0.5, "name_similarity": 0.4, "address_similarity": 0.3,
                  "meter_deviation": 0.3, "billing_deviation": 0.4, "anomaly_count": 3, "missing_fields": 2})
print(f"  Confidence: {r['confidence_score']}% | {r['prediction']} | Risk: {r['risk_level']}")

---
## 5. Save to Google Drive (Optional)

Mount Google Drive and copy output reports.

In [ ]:
if ON_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")

    # Copy all output reports to Drive
    import shutil
    drive_output = "/content/drive/MyDrive/PLN_Crosscheck_Reports"
    os.makedirs(drive_output, exist_ok=True)

    for f in os.listdir(OUTPUT_DIR):
        if f.endswith((".xlsx", ".pdf")):
            src = os.path.join(OUTPUT_DIR, f)
            dst = os.path.join(drive_output, f)
            shutil.copy2(src, dst)
            print(f"Copied: {f}")

    print(f"\nAll reports saved to: {drive_output}")
else:
    print(f"Reports are in: {OUTPUT_DIR}")